# 对抗示例生成

[![](https://gitee.com/mindspore/docs/raw/r1.3/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/r1.3/tutorials/source_zh_cn/intermediate/image_and_video/adversarial_example_generation.ipynb)

近年来随着数据、计算能力、理论的不断发展演进，深度学习在图像、文本、语音、自动驾驶等众多领域都得到了广泛应用。与此同时，人们也越来越关注各类模型在使用过程中的安全问题，因为AI模型很容易受到外界有意无意的攻击而产生错误的结果。在本案例中，我们将以梯度符号攻击FGSM（Fast Gradient Sign Method）为例，演示此类攻击是如何误导模型的。

## 对抗样本定义

Szegedy在2013年最早提出对抗样本的概念：在原始样本处加入人类无法察觉的微小扰动，使得深度模型性能下降，这种样本即对抗样本。如下图所示，本来预测为“panda”的图像在添加噪声之后，模型就将其预测为“gibbon”，右边的样本就是一个对抗样本：

![fgsm-panda-image](images/panda.png)

> 图片来自[Explaining and Harnessing Adversarial Examples](https://arxiv.org/abs/1412.6572)。


## 攻击方法

对模型的攻击方法可以按照以下方法分类：

1. 攻击者掌握的信息多少：

    1.1 白盒攻击：攻击者具有对模型的全部知识和访问权限，包括模型结构、权重、输入、输出。攻击者在产生对抗性攻击数据的过程中能够与模型系统有所交互。攻击者可以针对被攻击模型的特性设计特定的攻击算法。

    1.2 黑盒攻击：与白盒攻击相反，攻击者仅具有关于模型的有限知识。攻击者对模型的结构权重一无所知，仅了解部分输入输出。

2. 攻击者的目的：

    2.1 有目标的攻击：攻击者将模型结果误导为特定分类。

    2.2 无目标的攻击：攻击者只想产生错误结果，而不在乎新结果是什么。

本案例中用到的FGSM是一种白盒攻击方法，既可以是有目标也可以是无目标攻击。

更多的模型安全功能可参考[MindArmour](https://www.mindspore.cn/mindarmour)，现支持FGSM、LLC、Substitute Attack等多种对抗样本生成方法，并提供对抗样本鲁棒性模块、Fuzz Testing模块、隐私保护与评估模块，帮助用户增强模型安全性。

### 梯度符号攻击（FGSM）

正常分类网络的训练会定义一个损失函数，用于衡量模型输出值与样本真实标签的距离，通过反向传播计算模型梯度，梯度下降更新网络参数，减小损失值，提升模型精度。

FGSM（Fast Gradient Sign Method）是一种简单高效的对抗样本生成方法。不同于正常分类网络的训练过程，FGSM计算loss对于输入的梯度$\nabla_x J(\theta ,x ,y)$，这个梯度表征了loss对于输入变化的敏感性。然后在原始输入加上上述梯度，使得loss值增大，模型对于改造后的输入样本分类效果变差，达到攻击效果。对抗样本的另一要求是生成样本与原始样本的差异要尽可能的小，使用sign函数可以使得修改图片时尽可能的均匀。

产生的对抗扰动用公式可以表示为：

$$ \eta  = \varepsilon  sign(\nabla_x  J(\theta))$$

对抗样本的产生可公式化为：

$$ perturbed\_image = image + epsilon \times sign(data\_grad) = x + \epsilon \times sign(\nabla_x J(\theta ,x ,y)) $$

其中，

- $x$：正确分类为“Pandas”的原始输入图像。
- $y$：是$x$的输出。
- $\theta$：模型参数。
- $\varepsilon$：攻击系数。
- $J(\theta, x, y)$：训练网络的损失。
- $\nabla_x  J(\theta)$：反向传播梯度。

## 实现攻击

本案例将使用MNIST训练一个精度达标的LeNet网络，然后运行上文中所提到的FGSM攻击方法，实现错误分类的效果。

首先导入模型训练需要的库

In [ ]:
# 导入模型训练需要的库
import os
import numpy as np

from mindspore import Tensor, context, Model, load_checkpoint, load_param_into_net
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore.common.initializer import Normal
from mindspore.train.callback import LossMonitor
import mindspore.dataset as ds
import mindspore.dataset.transforms.c_transforms as C
import mindspore.dataset.vision.c_transforms as CV
from mindspore.dataset.vision import Inter
from mindspore import dtype as mstype
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig

采用`GRAPH_MODE`在CPU/GPU/Ascend中运行本案例：

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target='Ascend')

### 训练LeNet网络

实验中使用LeNet作为演示模型完成图像分类，这里先定义网络并使用MNIST数据集进行训练。

定义LeNet网络：

In [3]:
class LeNet5(nn.Cell):

    def __init__(self, num_class=10, num_channel=1):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(num_channel, 6, 5, pad_mode='valid')
        self.conv2 = nn.Conv2d(6, 16, 5, pad_mode='valid')
        self.fc1 = nn.Dense(16 * 5 * 5, 120, weight_init=Normal(0.02))
        self.fc2 = nn.Dense(120, 84, weight_init=Normal(0.02))
        self.fc3 = nn.Dense(84, num_class, weight_init=Normal(0.02))
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    def construct(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

net = LeNet5()

下载MINIST数据集：

In [ ]:
!mkdir -p ./datasets/MNIST_Data/train ./datasets/MNIST_Data/test
!wget -NP ./datasets/MNIST_Data/train https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/train-labels-idx1-ubyte
!wget -NP ./datasets/MNIST_Data/train https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/train-images-idx3-ubyte
!wget -NP ./datasets/MNIST_Data/test https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/t10k-labels-idx1-ubyte
!wget -NP ./datasets/MNIST_Data/test https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/t10k-images-idx3-ubyte
!tree ./datasets/MNIST_Data

进行数据处理：

In [5]:
def create_dataset(data_path, batch_size=1, repeat_size=1,
                   num_parallel_workers=1):

    # 定义数据集
    mnist_ds = ds.MnistDataset(data_path)
    resize_height, resize_width = 32, 32
    rescale = 1.0 / 255.0
    shift = 0.0
    rescale_nml = 1 / 0.3081
    shift_nml = -1 * 0.1307 / 0.3081

    # 定义所需要操作的map映射
    resize_op = CV.Resize((resize_height, resize_width), interpolation=Inter.LINEAR)
    rescale_nml_op = CV.Rescale(rescale_nml, shift_nml)
    rescale_op = CV.Rescale(rescale, shift)
    hwc2chw_op = CV.HWC2CHW()
    type_cast_op = C.TypeCast(mstype.int32)

    # 使用map映射函数，将数据操作应用到数据集
    mnist_ds = mnist_ds.map(operations=type_cast_op, input_columns="label", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=resize_op, input_columns="image", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=rescale_op, input_columns="image", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=rescale_nml_op, input_columns="image", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=hwc2chw_op, input_columns="image", num_parallel_workers=num_parallel_workers)

    # 进行shuffle、batch操作
    buffer_size = 10000
    mnist_ds = mnist_ds.shuffle(buffer_size=buffer_size)
    mnist_ds = mnist_ds.batch(batch_size, drop_remainder=True)

    return mnist_ds

定义优化器与损失函数：

In [6]:
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
net_opt = nn.Momentum(net.trainable_params(), learning_rate=0.01, momentum=0.9)

定义网络参数：

In [7]:
config_ck = CheckpointConfig(save_checkpoint_steps=1875, keep_checkpoint_max=10)
ckpoint = ModelCheckpoint(prefix="checkpoint_lenet", config=config_ck)

定义LeNet网络的训练函数和测试函数：

In [8]:
def test_net(model, data_path):
    ds_eval = create_dataset(os.path.join(data_path, "test"))
    acc = model.eval(ds_eval, dataset_sink_mode=False)
    print("{}".format(acc))
    
def train_net(model, epoch_size, data_path, repeat_size, ckpoint_cb, sink_mode):
    ds_train = create_dataset(os.path.join(data_path, "train"), 32, repeat_size)
    model.train(epoch_size, ds_train, callbacks=[ckpoint_cb, LossMonitor(125)], dataset_sink_mode=sink_mode)

train_epoch = 1
mnist_path = "./datasets/MNIST_Data/"
dataset_size = 1
model = Model(net, net_loss, net_opt, metrics={"Accuracy": nn.Accuracy()})

训练LeNet网络：

In [9]:
train_net(model, train_epoch, mnist_path, dataset_size, ckpoint, False)

epoch: 1 step: 125, loss is 2.2939456
epoch: 1 step: 250, loss is 2.3160274
epoch: 1 step: 375, loss is 2.2910666
epoch: 1 step: 500, loss is 2.3069649
epoch: 1 step: 625, loss is 2.3095846
epoch: 1 step: 750, loss is 2.294733
epoch: 1 step: 875, loss is 1.8178409
epoch: 1 step: 1000, loss is 0.70218444
epoch: 1 step: 1125, loss is 0.10911212
epoch: 1 step: 1250, loss is 0.31053308
epoch: 1 step: 1375, loss is 0.03824125
epoch: 1 step: 1500, loss is 0.20801516
epoch: 1 step: 1625, loss is 0.08320282
epoch: 1 step: 1750, loss is 0.33072343


epoch: 1 step: 1875, loss is 0.15156953


测试此时的网络，可以观察到LeNet已经达到比较高的精度：

In [10]:
test_net(model, mnist_path)

{'Accuracy': 0.9602}


加载此时的网络参数：

In [ ]:
param_dict = load_checkpoint("checkpoint_lenet-1_1875.ckpt")
load_param_into_net(net, param_dict)

### 实现FGSM

在得到精准的LeNet网络之后，下面将会采用FSGM攻击方法，在图像中加载噪声后重新进行测试。首先根据上文中提到的公式定义FGSM攻击函数`attack`。

$$ \eta  = \varepsilon  sign(\nabla_x  J(\theta))$$

$$ perturbed\_image = image + epsilon \times sign(data\_grad) = x + \epsilon \times sign(\nabla_x J(\theta ,x ,y)) $$

`attcak`中图像`image`、攻击系数`epsilon`和反向梯度`data_grad`都来自外部输入：

In [12]:
def attack(image, epsilon, data_grad):
    sign = ops.Sign()
    sign_data_grad = sign(data_grad)
    perturbed_image = image + epsilon * sign_data_grad
    return perturbed_image

### 测试攻击效果

现在使用定义好的`attack`函数重新对网络进行测试：

In [13]:
def FGSMTest(net, ds_test, epsilon):
    # 使用Model接口包装模型
    pre_model = Model(net, net_loss, net_opt, metrics={"Accuracy": nn.Accuracy()})
    correct = 0
    incorrect = 0
    # 重新对ds_test测试集中的图片进行测试
    for data in ds_test:
        label = data["label"].asnumpy()
        image_tensor = Tensor(data['image'])
        pre_output = pre_model.predict(image_tensor)
        predicted = np.argmax(pre_output.asnumpy(), axis=1)
        if predicted[0] != label[0]:
            continue
        # 获得反向梯度
        grad = ops.composite.GradOperation()
        net_with_criterion = nn.WithLossCell(net, net_loss)
        image_grad = grad(net_with_criterion)(image_tensor, Tensor(data['label']))
        # 调用攻击函数实现攻击
        perturbed_data = attack(Tensor(data['image']), epsilon, image_grad)
        # 统计攻击后的精确度
        perturbed_predict = pre_model.predict(perturbed_data)
        perturbed_result = np.argmax(perturbed_predict.asnumpy(), axis=1)
        if perturbed_result[0] == label[0]:
            correct += 1
        if perturbed_result[0] != label[0]:
            incorrect += 1
    perturbed_accuracy = correct / (correct + incorrect)
    print ("Accuracy = ", perturbed_accuracy)
    return perturbed_accuracy

## 运行攻击

由FGSM攻击公式中可以看出，攻击系数$\varepsilon$越大，对梯度的改变就越大。当$\varepsilon$ 为零时则攻击效果不体现。

$$ \eta  = \varepsilon  sign(\nabla_x  J(\theta)) $$

现在，先观察当$\varepsilon$ = 0 时的测试结果：

In [14]:
# 定义测试数据集，batch_size=1表示每次取出一张图片
ds_test = create_dataset(os.path.join(mnist_path, "test"), batch_size=1).create_dict_iterator()
# 运行攻击
epsilon = 0
final_acc = FGSMTest(net, ds_test, epsilon)

Accuracy =  1.0


设定$\varepsilon$ = 0.8，再次观察攻击效果：

In [15]:
epsilon = 0.8
final_acc = FGSMTest(net, ds_test, epsilon)

Accuracy =  0.10975993290701332


可以看到，此时LeNet模型的精度大幅降低。以直观的图片为例，右侧图片与左侧图片几乎没有明显变化，但是FGSM方法成功误导了模型，使模型将其错误地归属到其他类别中。

![attack-result](images/attack_result.png)
